In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import os
print(os.getcwd())

os.chdir('gdrive/My Drive')
!ls

/content
 134___08
 Andrews
 ASPEN
'Assignment 2 CH4220.docx'
'Assignment 2 Question 1'
'Assignment 3 '
 Baahubali
'Books For Tia'
'BU8401 Assignment 2(1).docx'
'BU8401 Assignment 2.docx'
'BU8401 Assignment 2 Final.docx'
'BU8401 Case Study Problem 1 Pg 543'
'CH4220 Assignment 3 Final (1).docx'
'CH4220 Assignment 3 Final.docx'
'CH4220 Assignment 5.docx'
'Chapter 4.docx'
'Colab Notebooks'
'Copy of Group 11.gslides'
'Copy of NavCon Project Report - v0.1.docx'
'Copy of Trip at Us Project Report (1).gdoc'
'Copy of Trip at Us Project Report (2).gdoc'
'Copy of Trip at Us Project Report (3).gdoc'
'Copy of Trip at Us Project Report.gdoc'
'Copy of VC Pitch Presentation.pptx.gslides'
'DS101 Data Science'
'EZCHORES Presentation.gslides'
 ezgif.com-video-to-gif.gif
'Financial Markets (Singapore).gsheet'
'FYDP Assignment 1 (FINAL)[118358].pdf'
'FYDP Part 2'
'FYP Assignment 1.docx'
'FYP Assignment 2 Final.docx'
'Getting started.pdf'
 GRE
'Green Chemistry notes (1).docx'
'Hackwagon Course 1.zip'
'Intr

In [ ]:
!pip install -U --pre tensorflow=="2.2.0"

     |████████████████████████████████| 516.2MB 32kB/s 
     |████████████████████████████████| 460kB 49.5MB/s 
     |████████████████████████████████| 3.0MB 45.7MB/s 
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninstalled tensorboard-2.3.0
  Found existing installation: tensorflow 2.3.0
    Uninstalling tensorflow-2.3.0:
      Successfully uninstalled tensorflow-2.3.0


In [ ]:
import os
import pathlib

print(os.getcwd())

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

/content/gdrive/My Drive


In [ ]:
%%bash
cd models/research/
protoc object_detection/protos/*.proto --python_out=.
cp object_detection/packages/tf2/setup.py .
python -m pip install .

In [ ]:
%%bash 
cd models/research
pip install .

Processing /content/gdrive/My Drive/models/research
  Created wheel for object-detection: filename=object_detection-0.1-cp36-none-any.whl size=1568028 sha256=2a7a23aee86fe9fac2bb3dd1373e3a46d49b96bb650c035be7f016d08420b63c
  Stored in directory: /tmp/pip-ephem-wheel-cache-8501sfoi/wheels/39/17/33/c621ad7a9ed7adb0845f36fc4a166f6c0d3e7b58c93ad05876
  Created wheel for avro-python3: filename=avro_python3-1.10.0-cp36-none-any.whl size=43735 sha256=afe5e8b3634e4caf01f1aa195ea96f3092d380e37b1d9307d9026e61ab145946
  Stored in directory: /root/.cache/pip/wheels/3f/15/cd/fe4ec8b88c130393464703ee8111e2cddebdc40e1b59ea85e9
  Created wheel for py-cpuinfo: filename=py_cpuinfo-7.0.0-cp36-none-any.whl size=20071 sha256=e0d09bada9c6ecf91462b9f97d6d045a971e62a5003079f93c62ebae7d8917b6
  Stored in directory: /root/.cache/pip/wheels/f1/93/7b/127daf0c3a5a49feb2fecd468d508067c733fba5192f726ad1
Successfully built object-detection avro-python3 py-cpuinfo
  Found existing installation: tensorboard 2.2.2
    U

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [ ]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [ ]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

### Testing the SSD MobileNet V2 

In [ ]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))

  return model

In [ ]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images/CocoDataSet_ImageObjectDetection/Outdoor')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpeg")))
TEST_IMAGE_PATHS

[PosixPath('models/research/object_detection/test_images/CocoDataSet_ImageObjectDetection/Outdoor/StopSign.jpeg')]

In [ ]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

76537856/76534733 [==============================] - 1s 0us/step
INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [ ]:
print(detection_model.signatures['serving_default'].inputs)

[<tf.Tensor 'image_tensor:0' shape=(None, None, None, 3) dtype=uint8>]


In [ ]:
detection_model.signatures['serving_default'].output_dtypes

{'detection_boxes': tf.float32,
 'detection_classes': tf.float32,
 'detection_scores': tf.float32,
 'num_detections': tf.float32}

In [ ]:
detection_model.signatures['serving_default'].output_shapes

{'detection_boxes': TensorShape([None, 100, 4]),
 'detection_classes': TensorShape([None, 100]),
 'detection_scores': TensorShape([None, 100]),
 'num_detections': TensorShape([None])}

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [ ]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))

  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)
  #image_np = cv2.resize(image_np, (750,600))
  return image_np
  #display(Image.fromarray(image_np))

In [ ]:
import cv2
for index, image_path in enumerate(TEST_IMAGE_PATHS):
  image = show_inference(detection_model, image_path)
  path = '/content/gdrive/My Drive/models/research/object_detection/test_images/' + 'image' + str(index) + '.jpg'
  print(path)
  cv2.imwrite(path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

/content/gdrive/My Drive/models/research/object_detection/test_images/image0.jpg
